In [1]:
import networkx as nx
import pickle
import matplotlib.pyplot as plt
from sklearn import cluster
import networkx as nx
from collections import defaultdict
import matplotlib.pyplot as plt
from matplotlib import cm
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.metrics.cluster import normalized_mutual_info_score
from sklearn.metrics.cluster import adjusted_rand_score

## Compute vessel sum per route

In [2]:
# step 1: compute most completed trips and link to nodes
df_2021 = pd.read_csv('data/2021/2021_combined.csv')
all_trips_ranked = df_2021.groupby(by=['UNLO_herkomst', 'UNLO_bestemming']).count().reset_index().sort_values(
    by='Jaarmaand', ascending=False).iloc[:, :3].reset_index(drop=True)
# make dict to store trips
trip_dict = {}

In [3]:
# now fill this dict
for i in range(len(all_trips_ranked) - 1):
    origin = all_trips_ranked['UNLO_herkomst'][i]
    destination = all_trips_ranked['UNLO_bestemming'][i]
    trip_count = all_trips_ranked['Unnamed: 0'][i]

    if (origin, destination) in trip_dict.keys():
        trip_dict[(origin, destination)] += trip_count
    elif (destination, origin) in trip_dict.keys():
        trip_dict[(destination, origin)] += trip_count
    else:
        trip_dict[(origin, destination)] = trip_count

In [4]:
# now sort values
trip_dict = dict(sorted(trip_dict.items(), key=lambda item: item[1], reverse=True))
trip_dict_df = {'origin': [], 'destination': [], 'trip_count': []}
for key, value in trip_dict.items():
    trip_dict_df['origin'].append(key[0])
    trip_dict_df['destination'].append(key[1])
    trip_dict_df['trip_count'].append(value)


In [5]:
df_common_trips = pd.DataFrame.from_dict(trip_dict_df)


## Get and clean harbour data NL, Ger and Bel

In [6]:


## Now get coordinates harbours
pd.read_csv('data/harbour_codes_coords/2021-2 UNLOCODE CodeListPart1.csv', encoding="ISO-8859-1", header=None, sep=",",
            index_col=None)

,0,1,2,3,4,5,6,7,8,9,10,11
0,NaN,AD,NaN,.ANDORRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,AD,ALV,Andorra la Vella,Andorra la Vella,NaN,--34-6--,AI,601.0,NaN,4230N 00131E,NaN
2,NaN,AD,CAN,Canillo,Canillo,NaN,--3-----,RL,307.0,NaN,4234N 00135E,NaN
3,NaN,AD,ENC,Encamp,Encamp,NaN,--3-----,RL,307.0,NaN,4232N 00134E,NaN
4,NaN,AD,ESC,Escaldes-Engordany,Escaldes-Engordany,NaN,--3-----,RL,307.0,NaN,4231N 00133E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
54611,NaN,FR,ZTG,Zetting,Zetting,57,--3-----,RL,1901.0,NaN,4905N 00708E,NaN
54612,NaN,FR,ZL2,Zilia,Zilia,2B,-----6--,RL,1901.0,NaN,4231N 00854E,NaN
54613,NaN,FR,ZWL,Zinswiller,Zinswiller,67,--3-----,RL,1901.0,NaN,4855N 00735E,NaN
54614,NaN,FR,PCZ,Zoteux,Zoteux,62,-----6--,RL,1001.0,NaN,5037N 00153E,NaN


In [7]:
#first identify all files in 2021 folder and save their names
from os import walk

f = []
for (dirpath, dirnames, filenames) in walk('data/harbour_codes_coords'):
    f.extend(filenames)
    break

In [8]:
d_type_dict = {}
for i in range(12):
    d_type_dict[i] = str

In [9]:

#empty list to store data
li = []

#read everything in as object for now
for filename in f:
    df = pd.read_csv("data/harbour_codes_coords/" + str(filename), sep=',', index_col=None, header=None,
                     encoding="ISO-8859-1", dtype = d_type_dict)
    print(filename, 'handled')
    li.append(df)

2021-2 UNLOCODE CodeListPart1.csv handled
2021-2 UNLOCODE CodeListPart2.csv handled
2021-2 UNLOCODE CodeListPart3.csv handled


In [10]:
#create df
df_harbour_codes = pd.concat(li, axis=0, ignore_index=True)
df_benelux = df_harbour_codes.loc[
    (df_harbour_codes[1] == 'NL') | (df_harbour_codes[1] == 'DE') | (df_harbour_codes[1] == 'BE') | (
                df_harbour_codes[1] == 'LU')]

In [11]:
df_benelux.fillna(np.nan, inplace=True)

C:\Users\Fabia\AppData\Local\Temp\ipykernel_23356\2771679216.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benelux.fillna(np.nan, inplace=True)


In [12]:
# fix layout and column names
# now drop empty columns
df_benelux.rename(columns={0: 'mutation', 1: 'country', 2: 'city_abbr', 3: 'city_full', 4: 'city_full_2', 5: 'subdivision', 6: 'function', 7:'status', 8:'date', 9:'iata_code', 10: 'coords', 11: 'comments'}, inplace=True)

C:\Users\Fabia\AppData\Local\Temp\ipykernel_23356\2598630862.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_benelux.rename(columns={0: 'mutation', 1: 'country', 2: 'city_abbr', 3: 'city_full', 4: 'city_full_2', 5: 'subdivision', 6: 'function', 7:'status', 8:'date', 9:'iata_code', 10: 'coords', 11: 'comments'}, inplace=True)


In [13]:
df_benelux

,mutation,country,city_abbr,city_full,city_full_2,subdivision,function,status,date,iata_code,coords,comments
5129,NaN,BE,NaN,.BELGIUM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5130,NaN,BE,ALB,Aalbeke,Aalbeke,VWV,-----6--,RL,0601,NaN,5046N 00313E,NaN
5131,NaN,BE,AAB,Aalst,Aalst,VOV,1-3-----,RN,0307,NaN,5056N 00402E,NaN
5132,NaN,BE,AAL,Aalter,Aalter,VOV,1-3-----,RN,0307,NaN,5105N 00327E,NaN
5133,NaN,BE,AAS,Aarschot,Aarschot,VBR,--3-----,RN,0307,NaN,5059N 00450E,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
80512,NaN,NL,ZWS,Zwartsluis,Zwartsluis,NaN,1-3-----,RL,9811,NaN,NaN,NaN
80513,NaN,NL,ZGE,Zwiggelte,Zwiggelte,DR,--3-----,RL,0407,NaN,5252N 00635E,NaN
80514,NaN,NL,ZWI,Zwijndrecht,Zwijndrecht,NaN,123-----,AF,9602,NaN,NaN,NaN
80515,NaN,NL,ZWN,Zwinderen,Zwinderen,DR,1-3-----,RN,0501,NaN,5244N 00641E,NaN


In [14]:
df_benelux = df_benelux.loc[:,['country','city_abbr','function', 'coords']]

In [15]:
# df_benelux.country = df_benelux.country.to_string()
# df_benelux.city_abbr = df_benelux.city_abbr.to_string()
# df_benelux.function = df_benelux.function.to_string()
# df_benelux.coords = df_benelux.coords.to_string()

In [16]:
df_benelux

,country,city_abbr,function,coords
5129,BE,NaN,NaN,NaN
5130,BE,ALB,-----6--,5046N 00313E
5131,BE,AAB,1-3-----,5056N 00402E
5132,BE,AAL,1-3-----,5105N 00327E
5133,BE,AAS,--3-----,5059N 00450E
...,...,...,...,...
80512,NL,ZWS,1-3-----,NaN
80513,NL,ZGE,--3-----,5252N 00635E
80514,NL,ZWI,123-----,NaN
80515,NL,ZWN,1-3-----,5244N 00641E


In [18]:
df_benelux.dtypes

country      object
city_abbr    object
function     object
coords       object
dtype: object

In [17]:
df_benelux.loc[(df_benelux.country.notnull()) and (df_benelux.city_abbr.notnull())]

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [ ]:
df_benelux.dtypes